In [ ]:
import os
os.chdir('..')

In [ ]:
import torch
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel

In [ ]:
baseline_path = 'results/transfer/baseline.pt'
paths = {
    'glofnd': 'results/transfer/glofnd.pt',
    'fnd': 'results/transfer/fnd.pt',
}

In [ ]:
baseline = torch.load(baseline_path)
baseline_means = {k: np.mean(v).round(2) if isinstance(v, list) else v for k, v in baseline.items()}
baseline_d = {k: f"{np.mean(v).round(2)} ({np.std(v).round(2)})" if isinstance(v, list) else v for k, v in baseline.items()}
baseline_d['name'] = 'baseline'
baseline_d['avg'] = np.mean(list(baseline_means.values())[:-1])

In [ ]:
baseline_d

In [ ]:
x_means = [baseline_d]
x_p = []
for name, path in paths.items():
    x = torch.load(path, weights_only=False)

    # means = {k: np.mean(v).round(2) if isinstance(v, list) else v for k, v in values.items()}
    means = {}
    p_values = {}
    means['name'] = name
    
    avg = []
    for k, v in x.items():
        if isinstance(v, list):
            p = round(ttest_rel(v, baseline[k], nan_policy='raise', alternative='greater')[1], 3)
            # means[k] = f"{np.mean(v).round(2)} ({np.std(v).round(2)}) (p={p})"
            means.append({k: f"{np.mean(v).round(2)} ({np.std(v).round(2)})"})
            p_values.append({k: p})

            avg.append(np.mean(v))
        else:
            means[k] = v
            p_values[k] = v
    
    means['avg'] = np.mean(avg).round(2)
    x_means.append(means)
    x_p.append(p_values)

In [ ]:
df_means = pd.DataFrame(x_means).sort_values('avg', ascending=False)
df_means

In [ ]:
df_p = pd.DataFrame(x_p)

In [ ]:
df_means.to_latex('results/transfer/means.tex', index=False)